In [16]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR,SVC
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer

In [7]:
df = pd.read_excel("../数据清洗整理/训练数据2(加入对手特征).xlsx")
df.head()

,GAME_ID,PCT_UAST_3PM,AST,PCT_AST_FGM,FG3M,DREB,FGM,PFD,PCT_PTS_3PT,PTS_2ND_CHANCE,...,AST_TO_CHANGE_OPP,REB_PCT_CHANGE_OPP,PTS_PAINT_CHANGE_OPP,FTA_CHANGE_OPP,PACE_CHANGE_OPP,BLKA_CHANGE_OPP,BLK_CHANGE_OPP,PCT_PTS_PAINT_CHANGE_OPP,PCT_FGA_3PT_CHANGE_OPP,PCT_PTS_2PT_MR_CHANGE_OPP
0,21600075,0.0236,5.2,0.0356,3.6,2.2,6.4,5.8,0.0682,-1.2,...,0.676,0.0028,0.0,-4.2,1.102,0.2,2.2,-0.0432,0.1124,-0.0104
1,21600075,-0.0700,0.4,-0.0122,2.6,9.8,1.2,-1.6,0.0572,5.8,...,-0.582,0.0602,2.8,-2.0,0.826,0.4,-0.8,-0.0114,0.0194,-0.0030
2,21600077,-0.1200,-4.6,-0.1186,-2.8,6.2,-0.6,-1.6,-0.0684,6.8,...,0.000,-0.0126,6.0,8.4,8.704,-0.8,-0.4,0.0402,-0.0408,0.0062
3,21600077,0.2154,-3.2,-0.0282,1.4,-0.2,-4.6,5.8,0.0850,-2.0,...,0.480,-0.0696,5.2,-0.6,3.388,-0.2,-0.2,0.0620,0.0136,-0.0142
4,21600078,0.0414,-3.2,-0.0140,-4.6,-3.4,-4.0,-2.2,-0.0924,5.8,...,0.704,-0.0364,-2.4,2.2,-2.772,-0.4,2.6,0.0084,-0.0124,0.0040


In [8]:
exclude_cols = ["GAME_ID","PTS","NEXT_CHANGE","TEAM_NAME",'PTS_CHANGE_OPP']
columns = [col for col in df.columns if col not in exclude_cols]
X=df[columns].values
y=df['NEXT_CHANGE'].values
y2=Binarizer(0).transform([y])[0]
standard = StandardScaler()
standard.fit(X)
X=standard.transform(X)
X.shape

(2270, 106)

In [9]:
split_num = round(X.shape[0]*0.8)
X_train = X[:split_num]
y_train = y2[:split_num]
X_test = X[split_num:]
y_test = y2[split_num:]
##########
clf = LogisticRegressionCV()
clf2=GridSearchCV(KNeighborsClassifier(),{'n_neighbors':[5,10,15]})
clf3 = GridSearchCV(SVC(),{'C':[1,2,3]})
clf4 = BaggingClassifier(DecisionTreeClassifier(),200)
###############
clf.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
clf4.fit(X_train,y_train)
clf.score(X_test,y_test),clf2.score(X_test,y_test),clf3.score(X_test,y_test),clf4.score(X_test,y_test)

(0.74008810572687223,
 0.69603524229074887,
 0.72026431718061679,
 0.74229074889867841)

In [15]:
clf4 = BaggingClassifier(DecisionTreeClassifier(),200)
clf4.fit(X_train,y_train)
clf4.score(X_test,y_test)

0.74669603524229078

In [21]:
clf5 = RandomForestClassifier(200,min_samples_split=2,min_samples_leaf=2)
clf5.fit(X_train,y_train)
clf5.score(X_test,y_test)

0.74449339207048459

In [22]:
clf5.score(X,y2)

0.94889867841409692

In [11]:
clf5 = AdaBoostClassifier(n_estimators=500,learning_rate=1)
clf5.fit(X_train,y_train)
clf5.score(X_test,y_test)

0.68061674008810569